In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = "/home/group07/.conda/envs/group07/bin/python3"
os.environ['PYSPARK_DRIVER_PYTHON'] = "/home/group07/.conda/envs/group07/bin/python3"

In [2]:
print(os.environ['PYSPARK_DRIVER_PYTHON'])

/home/group07/.conda/envs/group07/bin/python3


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [4]:
##Configuration

#config the connector jar file
spark = (SparkSession.builder.appName("SimpleSparkJob").master("spark://34.142.194.212:7077")
         .config("spark.jars", "/opt/spark/jars/gcs-connector-latest-hadoop2.jar")
         .config("spark.executor.memory", "2G")  #excutor excute only 2G
        .config("spark.driver.memory","4G") 
        .config("spark.executor.cores","3") #Cluster use only 3 cores to excute
        .config("spark.python.worker.memory","1G") # each worker use 1G to excute
        .config("spark.driver.maxResultSize","3G") #Maximum size of result is 3G
        .config("spark.kryoserializer.buffer.max","1024M")
         .getOrCreate())
#config the credential to identify the google cloud hadoop file 
spark.conf.set("google.cloud.auth.service.account.json.keyfile","/opt/spark/lucky-wall-393304-2a6a3df38253.json")
spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
spark._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.enable', 'true')

## Connect to the file in Google Bucket with Spark


# spark.show()

# spark.stop # Ending spark job



23/12/08 13:18:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
path=f"gs://it4043e-it5384/it4043e/it4043e_group7_problem1/final"
tweet_final = spark.read \
    .format("parquet") \
    .option("inferSchema", "true") \
    .option("escape", "\"") \
    .option("multiline", "true") \
    .option("wholeFile", "true") \
.load(path)
tweet_final.show()

+--------------------+--------------------+-------------------+------------------+--------------------+------+-------------+--------------------+--------------------+
|            username|          tweet_text| datetime_timestamp|          tweet_id|     discussion_link|images|video_preview|       user_location|    user_description|
+--------------------+--------------------+-------------------+------------------+--------------------+------+-------------+--------------------+--------------------+
|                   0|$QNT #QNT Communi...|2021-08-05 15:56:26|158836031181224018|https://twitter.c...|    []|           []|                 USA|#BTC $ETH \n$QNT ...|
|                   0|@QNTMami Safe to ...|2021-11-26 17:14:11|146609917543097691|https://twitter.c...|    []|           []|                 USA|$QNT $HTR $LCX $C...|
|             0001337|@JackFarley96 It’...|2022-01-22 07:48:02|159052628733145591|https://twitter.c...|    []|           []|              SHA256|               🏴‍☠️|

In [6]:
from pyspark.sql.functions import isnan, when, count, col


print(tweet_final.where(tweet_final.datetime_timestamp.isNull()).count())
tweet_final.select([count(when(col(c).isNull(), c)).alias(c) for c in tweet_final.columns]).show() 

0


+--------+----------+------------------+--------+---------------+------+-------------+-------------+----------------+
|username|tweet_text|datetime_timestamp|tweet_id|discussion_link|images|video_preview|user_location|user_description|
+--------+----------+------------------+--------+---------------+------+-------------+-------------+----------------+
|    6798|         1|                 0|       0|             63|     0|            0|      1240033|          368521|
+--------+----------+------------------+--------+---------------+------+-------------+-------------+----------------+



In [7]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import isnan, when, count, col

non_ascii_pattern = "[^\x00-\x7F]"
# Apply text preprocessing
df_preprocessed = tweet_final.withColumn("tweet_text", regexp_replace(col("tweet_text"), non_ascii_pattern, "")).withColumn("user_description",  regexp_replace(col("user_description"), non_ascii_pattern, ""))

In [8]:
df_preprocessed.select("user_description").show(5,False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_description                                                                                                                                              |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|#BTC $ETH \n$QNT $CHR $HTR $LCX $RSR $ALGO\nNot Financial Advice #DYOR\n\n#lifestyle #TIMEFreedom #travel #boxing #investing #crypto #realestate              |
|$QNT $HTR $LCX $CHR $ETH #BTC $RSR\nNot Financial Advice \n\nAccumulate #Capital \nCreate #IncomeStreams\nGain #TimeFreedom \n\n#Travel #SelfEducation #Crypto|
|                                                                                                                                                              |
|#POW = e your own ank = Type 1 ci

In [9]:
from pyspark.sql.functions import split, col, substring
df_split = df_preprocessed.withColumn("date", split(col("datetime_timestamp"), " ")[0]).withColumn("time", split(col("datetime_timestamp"), " ")[1])

In [10]:
df_split.show(5)

+--------+--------------------+-------------------+------------------+--------------------+------+-------------+-------------+--------------------+----------+--------+
|username|          tweet_text| datetime_timestamp|          tweet_id|     discussion_link|images|video_preview|user_location|    user_description|      date|    time|
+--------+--------------------+-------------------+------------------+--------------------+------+-------------+-------------+--------------------+----------+--------+
|       0|$QNT #QNT Communi...|2021-08-05 15:56:26|158836031181224018|https://twitter.c...|    []|           []|          USA|#BTC $ETH \n$QNT ...|2021-08-05|15:56:26|
|       0|@QNTMami Safe to ...|2021-11-26 17:14:11|146609917543097691|https://twitter.c...|    []|           []|          USA|$QNT $HTR $LCX $C...|2021-11-26|17:14:11|
| 0001337|@JackFarley96 Its...|2022-01-22 07:48:02|159052628733145591|https://twitter.c...|    []|           []|       SHA256|                    |2022-01-22|07

In [11]:
from pyspark.sql.functions import udf
from language_detector import detect_language

# Define a Python function that wraps the language detection
def detect_language_udf(text):
    try:
        return detect_language(text)
    except:
        return None

# Register the Python function as a UDF
detect_language_pyspark_udf = udf(detect_language_udf)

# Assuming `df` is your DataFrame and `text_column` is the column containing text
df_result = df_split.withColumn("language_tweet", detect_language_pyspark_udf("tweet_text"))


In [12]:
df_result.show(5)

23/12/08 13:19:18 WARN TaskSetManager: Lost task 0.0 in stage 4.0 (TID 4) (10.148.0.23 executor 1): java.io.IOException: Cannot run program "/home/group07/.conda/envs/group07/bin/python3": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:239)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:139)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.sc

Py4JJavaError: An error occurred while calling o88.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 4 times, most recent failure: Lost task 0.3 in stage 4.0 (TID 7) (10.148.0.22 executor 0): java.io.IOException: Cannot run program "/home/group07/.conda/envs/group07/bin/python3": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:239)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:139)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.Batc

In [13]:
df_split.write.parquet("gs://it4043e-it5384/it4043e/it4043e_group7_problem1/final_preprocess",mode = "overwrite")

In [33]:
spark.stop()